In [ ]:
#cell type hierachy, identify cell types. 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline


data = pd.read_csv('zlpatch_TPM_n26.csv', index_col = 0)
gene_name = pd.read_table('GRCh38_genes_ENS.txt', header=0 ) #header, index_col
gene_name.rename(columns ={'Ensembl Gene ID':'gene','Associated Gene Name':'name'}, inplace=True)
df_real=pd.read_csv('zlpatch_TPM_n26.csv', index_col = 0)

target_genes = ['RELN','MKI67','OLIG2','GFAP','NES','SOX2','PAX2','S100B','GAD1','GAD2','DLX1','DLX2','CUX2','BCL11B','SATB2','VIM','RBFOX3','RORB','TLE4','TBR1']
xyxy =gene_name[gene_name['name'].isin(target_genes)]
xy = xyxy['gene'].tolist() 
print(xyxy)
print(xy)
#print(type(xy))

dfplot = df_real.loc[:,xy]

dfplot.columns=xyxy['name'].tolist()
dfplot = dfplot.reindex(columns=target_genes)
data =dfplot.copy()
data.head()

from sklearn.preprocessing import normalize
data_scaled = normalize(data)
data_scaled = pd.DataFrame(data_scaled, columns=data.columns)
data_scaled.head()

import scipy.cluster.hierarchy as shc
plt.figure(figsize=(10, 7))  
plt.title("Dendrograms")  
dend = shc.dendrogram(shc.linkage(data_scaled, method='ward'))

ax,fig = plt.subplots(figsize=(8,4))

plt.title("Dendrograms")  
dend = shc.dendrogram(shc.linkage(data_scaled, method='ward'))
plt.axhline(y=6, color='r', linestyle='-.')

plt.savefig('cluster_celltype_moreMarker.png', dpi = 300, bbox_inches = 'tight')

import seaborn as sns; sns.set(color_codes=True)

ax = sns.clustermap(data_scaled,method='ward',figsize=(8,8))
#plt.tick_params(axis="x", labelsize=28)
#plt.xlabel('SATB2',size=20)
plt.savefig('cluster_celltype_heat_moreMarker.png', dpi = 300, bbox_inches = 'tight')


from sklearn.cluster import AgglomerativeClustering
cluster = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward')  
cluster.fit_predict(data_scaled)

data.index[cluster.fit_predict(data_scaled)<1]


ax,figure = plt.subplots(figsize=(4,3))
ax = plt.scatter(data_scaled['SATB2'], data_scaled['GAD1'],c=cluster.labels_,marker='o',edgecolors='b') 
plt.xlabel('SATB2',size=20)
plt.ylabel('GAD1',size=20)

plt.savefig('satb2_gad1_cluster_moreMarker.png', dpi = 300, bbox_inches = 'tight')



data.head()

data.index[cluster.fit_predict(data_scaled)<1]
data['cell_type']= 'IN'

for x in data.index[cluster.fit_predict(data_scaled)<1]:
    data.loc[x,'cell_type']='EX'

#data.loc[:,['cell_type']].sort_values('cell_type')
#Index(['c13', 'c15', 'c16', 'c21', 'c22', 'c25', 'c26', 'c27', 'c28', 'c29', 'c30', 'c37', 'c41', 'c42', 'c43', 'c44']
#
data

data.to_csv('patchseq_celltype_moreMarker.csv')